# 🧠 Brain Connectome Analysis - One-Click Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Sean0418/Brain-Connectome/blob/main/notebooks/colab_demo.ipynb)

This notebook demonstrates the Brain Connectome analysis pipeline. Just click **Runtime → Run all** to execute the entire analysis!

## What this notebook does:
1. 📦 Installs the brain-connectome package
2. 📊 Loads sample HCP connectome data
3. 🔬 Runs sexual dimorphism analysis
4. 🤖 Trains ML classifiers (Random Forest + EBM)
5. 🔗 **Mediation Analysis**: Tests brain network mediation of cognitive-alcohol relationships
6. 📈 Visualizes the results

---


## Step 1: Setup Environment

First, we'll clone the repository and install dependencies. This takes about 2-3 minutes.


In [ ]:
# Install interpret FIRST (required for EBM) - must be before package import
%pip install -q interpret

# Clone or update the repository
import os

if os.path.exists("Brain-Connectome"):
    print("Repository exists, pulling latest changes...")
    %cd Brain-Connectome
    !git pull
else:
    print("Cloning repository...")
    !git clone https://github.com/Sean0418/Brain-Connectome.git
    %cd Brain-Connectome

# Install the package
%pip install -q -e .

print("✅ Setup complete!")

## Step 2: Load Data

We'll create sample data for demonstration. For your own analysis, you would mount Google Drive or upload your HCP data.


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

# Check if processed data exists, otherwise create sample data
data_path = Path("data/processed/full_data.csv")

if data_path.exists():
    print("Loading existing processed data...")
    data = pd.read_csv(data_path)
else:
    print("Creating sample data for demonstration...")
    # Create sample data that mimics HCP structure
    rng = np.random.default_rng(42)
    n_subjects = 200
    n_pcs = 60

    # Create sample data with gender differences in some PCs
    data = {"Subject": range(1, n_subjects + 1)}
    data["Gender"] = rng.choice(["M", "F"], n_subjects)

    # Add PC columns with some showing gender differences
    for i in range(1, n_pcs + 1):
        if i in [1, 3, 12, 23, 33]:  # These will show dimorphism
            male_mean = 0.5 if i % 2 == 0 else -0.5
            data[f"Struct_PC{i}"] = np.where(
                np.array(data["Gender"]) == "M",
                rng.normal(male_mean, 1, n_subjects),
                rng.normal(-male_mean, 1, n_subjects),
            )
        else:
            data[f"Struct_PC{i}"] = rng.normal(0, 1, n_subjects)

    data = pd.DataFrame(data)

print(f"\n📊 Dataset loaded: {data.shape[0]} subjects, {data.shape[1]} features")
print("\nGender distribution:")
print(data["Gender"].value_counts())
data.head()

## Step 3: Sexual Dimorphism Analysis

We'll analyze which brain connectivity features differ significantly between males and females.


In [ ]:
from brain_connectome.analysis import DimorphismAnalysis

# Run dimorphism analysis
analysis = DimorphismAnalysis(data, gender_column="Gender")

# Analyze all structural PCs
struct_pcs = [col for col in data.columns if col.startswith("Struct_PC")]
results = analysis.analyze(feature_columns=struct_pcs)

# Show results
n_significant = results["Significant"].sum()
print(f"\n🔬 Found {n_significant} significant features (FDR < 0.05)")
print("\n📋 Top 10 features by effect size:")
results.head(10)

In [ ]:
import matplotlib.pyplot as plt

# Plot effect sizes
fig, ax = plt.subplots(figsize=(10, 8))

top20 = results.head(20)
colors = ["#1f77b4" if d < 0 else "#d62728" for d in top20["Cohen_D"]]

ax.barh(range(len(top20)), top20["Cohen_D"].values, color=colors)
ax.set_yticks(range(len(top20)))
ax.set_yticklabels(top20["Feature"])
ax.set_xlabel("Cohen's D (Effect Size)")
ax.set_title("Sexual Dimorphism: Top 20 Features by Effect Size")
ax.axvline(0, color="black", linestyle="-", linewidth=0.5)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

print("\n📊 Blue bars: Feature is higher in females")
print("📊 Red bars: Feature is higher in males")

## Step 4: Machine Learning Classification

Now we'll train a Random Forest classifier to predict gender from brain connectivity features.


In [ ]:
from sklearn.model_selection import train_test_split

from brain_connectome.models import ConnectomeRandomForest

# Prepare features and labels
X = data[struct_pcs].values
y = (data["Gender"] == "M").astype(int).values

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {len(X_train)} subjects")
print(f"Test set: {len(X_test)} subjects")

# Train classifier
clf = ConnectomeRandomForest(n_estimators=200, random_state=42)
clf.fit(X_train, y_train, feature_names=struct_pcs)

# Evaluate
metrics = clf.evaluate(X_test, y_test)
print(f"\n🎯 Test Accuracy: {metrics['accuracy']:.2%}")

In [ ]:
# Get and plot top features
importance = clf.get_top_features(n=15)

fig, ax = plt.subplots(figsize=(10, 6))
top15 = importance.head(15).iloc[::-1]
colors = plt.colormaps["viridis"](np.linspace(0.3, 0.9, len(top15)))

ax.barh(top15["Feature"], top15["Importance"], color=colors)
ax.set_xlabel("Importance")
ax.set_title("Random Forest: Top 15 Features for Gender Classification")
plt.tight_layout()
plt.show()

print("\n📋 Top 10 most important features:")
importance.head(10)

## Step 5: EBM (Explainable Boosting Machine)

EBM is an interpretable ML model that provides both global and local explanations.


In [ ]:
from brain_connectome.models import ConnectomeEBM

# Train EBM
ebm = ConnectomeEBM(learning_rate=0.01, max_bins=32, max_leaves=3, interactions=0, random_state=42)
ebm.fit(X_train, y_train, feature_names=struct_pcs)

# Evaluate
ebm_metrics = ebm.evaluate(X_test, y_test)
print(f"🎯 EBM Accuracy: {ebm_metrics['accuracy']:.2%}")
print(f"📊 Random Forest Accuracy: {metrics['accuracy']:.2%}")

# Plot EBM feature importance
ebm_importance = ebm.get_top_features(n=15)
fig, ax = plt.subplots(figsize=(10, 6))
top15_ebm = ebm_importance.head(15).iloc[::-1]
ax.barh(top15_ebm["Feature"], top15_ebm["Importance"], color="#2ecc71")
ax.set_xlabel("Importance")
ax.set_title("EBM: Top 15 Features (Interpretable Model)")
plt.tight_layout()
plt.show()

## Step 6: Mediation Analysis

Test whether brain networks **mediate** the relationship between cognitive traits and alcohol outcomes, stratified by sex.

**Research Question**: *How do structural brain networks mediate the relationship between cognitive traits and alcohol dependence differently across sexes?*

```
Cognitive Traits (X) → Brain Networks (M) → Alcohol Outcomes (Y)
                              ↑
                         Sex (moderator)
```


In [ ]:
from brain_connectome.analysis import MediationAnalysis, SexStratifiedMediation

# Create sample data with mediation structure for demonstration
# In real analysis, you would use actual HCP cognitive and alcohol measures
rng = np.random.default_rng(42)

# Add cognitive score (simulated fluid intelligence)
data["CognitiveScore"] = rng.normal(100, 15, len(data))

# Add alcohol severity (influenced by both cognitive and brain, with sex differences)
# This simulates the mediation: Cognitive → Brain → Alcohol
brain_effect = data["Struct_PC1"].values
cognitive_effect = (data["CognitiveScore"].values - 100) / 15

# Different mediation strength by sex
sex_modifier = np.where(data["Gender"] == "M", 0.4, 0.2)
data["AlcoholScore"] = (
    0.3 * cognitive_effect +  # Direct effect
    sex_modifier * brain_effect +  # Mediated effect (stronger in males)
    rng.normal(0, 0.5, len(data))
)

print("📊 Sample mediation data created")
print(f"   Cognitive Score: mean={data['CognitiveScore'].mean():.1f}")
print(f"   Alcohol Score: mean={data['AlcoholScore'].mean():.2f}")


In [ ]:
# Run sex-stratified mediation analysis
# Model: CognitiveScore → Struct_PC1 → AlcoholScore
print("🔬 Running Sex-Stratified Mediation Analysis...")
print("   Model: Cognitive → Brain Network → Alcohol")
print()

mediation = SexStratifiedMediation(n_bootstrap=500, random_state=42)
result = mediation.fit(
    data=data,
    cognitive_col="CognitiveScore",
    brain_col="Struct_PC1",
    alcohol_col="AlcoholScore",
    sex_col="Gender",
)

# Display results
print("=" * 50)
print("MEDIATION RESULTS")
print("=" * 50)

print("\n👨 MALES:")
print(f"   Indirect effect (a×b): {result.male.indirect_effect:.4f}")
print(f"   95% CI: [{result.male.ci_low:.4f}, {result.male.ci_high:.4f}]")
print(f"   Significant: {'✅ Yes' if result.male.significant else '❌ No'}")

print("\n👩 FEMALES:")
print(f"   Indirect effect (a×b): {result.female.indirect_effect:.4f}")
print(f"   95% CI: [{result.female.ci_low:.4f}, {result.female.ci_high:.4f}]")
print(f"   Significant: {'✅ Yes' if result.female.significant else '❌ No'}")

print("\n⚖️ SEX DIFFERENCE:")
print(f"   Difference (M - F): {result.difference:.4f}")
print(f"   95% CI: [{result.diff_ci_low:.4f}, {result.diff_ci_high:.4f}]")
print(f"   Significant: {'✅ Yes' if result.diff_significant else '❌ No'}")


In [ ]:
# Visualize mediation comparison
fig, ax = plt.subplots(figsize=(8, 5))

# Bar positions
x = np.array([0, 1])
width = 0.6

# Data
effects = [result.male.indirect_effect, result.female.indirect_effect]
errors = [
    [result.male.indirect_effect - result.male.ci_low, 
     result.female.indirect_effect - result.female.ci_low],
    [result.male.ci_high - result.male.indirect_effect,
     result.female.ci_high - result.female.indirect_effect]
]

colors = ["#3498db", "#e74c3c"]
bars = ax.bar(x, effects, width, color=colors, edgecolor="black", linewidth=1.5)
ax.errorbar(x, effects, yerr=errors, fmt="none", color="black", capsize=5, capthick=2)

ax.set_xticks(x)
ax.set_xticklabels(["Males", "Females"], fontsize=12)
ax.set_ylabel("Indirect Effect (Mediation)", fontsize=12)
ax.set_title("Sex Differences in Brain Network Mediation\nCognitive → Brain → Alcohol", fontsize=14)
ax.axhline(0, color="black", linestyle="-", linewidth=0.5)

# Add significance stars
for i, (effect, sig) in enumerate(zip(effects, [result.male.significant, result.female.significant])):
    if sig:
        ax.annotate("*", (x[i], effect + 0.02), ha="center", fontsize=16, fontweight="bold")

plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("   - The indirect effect represents brain network mediation strength")
print("   - Error bars show 95% bootstrap confidence intervals")
print("   - * indicates significant mediation (CI excludes zero)")


## 📋 Summary

This notebook demonstrated the Brain Connectome analysis pipeline:

1. **Data Loading**: Loaded structural connectome PC features
2. **Dimorphism Analysis**: Identified sexually dimorphic brain connectivity patterns
3. **Random Forest**: Trained an ensemble classifier for gender prediction
4. **EBM**: Trained an interpretable boosting model
5. **Mediation Analysis**: Tested sex-stratified mediation (Cognitive → Brain → Alcohol)
6. **Visualization**: Created publication-ready plots

### Research Question Addressed

> *How do structural brain networks mediate the relationship between cognitive traits and alcohol dependence differently across sexes?*

### Next Steps

- **Use your own data**: Upload HCP data to Google Drive and mount it
- **Run full pipeline**: Use `!python Runners/run_pipeline.py` for complete analysis
- **Use Docker**: `docker pull ghcr.io/sean0418/brain-connectome:latest`

### Links

- 📦 [GitHub Repository](https://github.com/Sean0418/Brain-Connectome)
- 🐳 [Docker Image](https://ghcr.io/sean0418/brain-connectome)


In [ ]:
print("\n" + "=" * 60)
print("🎉 Analysis Complete!")
print("=" * 60)
print(f"\n📊 Analyzed {data.shape[0]} subjects")
print(f"🔬 Found {n_significant} significant dimorphic features")
print(f"🎯 Random Forest accuracy: {metrics['accuracy']:.2%}")
print(f"🎯 EBM accuracy: {ebm_metrics['accuracy']:.2%}")
print(f"🔗 Mediation sex difference: {result.difference:.4f} (sig: {result.diff_significant})")
print("\n⭐ Star us on GitHub: https://github.com/Sean0418/Brain-Connectome")